# Preprocessing of real data

This notebook allows to create hourly data with precipitation, streamflow, potential evapotranspiration and date from the following files:

- MeteoSwiss combiprecip product
- Hourly streamflow data
- Daily minimum, maximal and mean temperature to compute the daily potential evapotranspiration, which is then distributed evenly over 24 hours and saved at an hourly resolution.
- Daily precipitation and streamflow data is also used to fill some possible gaps in the hourly ones.

/!\ The Pyeto Python package should be downloaded from [here](https://github.com/woodcrafty/PyETo) if it is not already present in the root folder of your repository.

#### START by dowloading the file 'data.zip' using the link below and extract it. Save the resulting 'data' folder in the folder 'experiments'.

https://www.dropbox.com/scl/fi/mo4xg95ktj7yt09e1o78a/data.zip?rlkey=9m8e4nshv831g7q129c1vsy53&st=rgy8wp05&dl=0

This file contains the hydrologic and climatic timeseries for a single demonstration catchment (Minster river in Euthal/Rüti (canton Schwyz), BAFU data).

In [1]:
import numpy as np
import os
import pandas as pd
from datetime import datetime
import pickle
from tqdm import tqdm
from geopy.geocoders import Nominatim

from helpers_preprocessing import get_acronym, get_latitude, get_PET_hargreaves, process_precip, process_hydro, merging_dfs

ALL_GIS_IDs = ['44']  # Catchments of interest's GIS IDs (only one catchment here with ID #44)

In [2]:
# File paths with catchment(s) properties and fluxes
path_J = os.path.join('..', 'data', 'real_data', 'polygons_CH1903_LV95_area_weighted_combiprecip')  # fluxes for only one catchment right now
path_daily = os.path.join('..', 'data', 'real_data', 'Daily_Data')
path_Q  = os.path.join('..', 'data', 'real_data', 'hourly_streamflow')
file_catchprop = os.path.join('..', 'data', 'CH_Catchments_Geodata_MF_20221209.csv')

In [3]:
# Load catchment static properties
df_catchment_properties = pd.read_csv(file_catchprop, header=None,  engine='python')  # why full of NaNs? RM

# Save catchment properties headers
data_type = df_catchment_properties.iloc[0, :]  # category
description = df_catchment_properties.iloc[1, :]  # feature
IDs = df_catchment_properties.iloc[2, :]  # feature ID/label

df_catchment_properties = df_catchment_properties.drop([0, 1, 2])  # remove headers
rows_idx = list(df_catchment_properties.iloc[:, 0])

df_info = df_catchment_properties.iloc[:, [0, 9]]  # collect catchment ID and information on data quality
df_info.columns = ['GIS_ID', 'INFO']

# Translate categories from german to english and associate
# each category with its corresponding features
categories = ['Area', 'Quality', 'Response', 'Climate', 'Altitude', 'Slope', 'runoff accumulation', 
              'storage capacity', 'permeability', 'waterlogging', 'thoroughness', 'land use',
              'ground cover' ] + 5*['Geology'] + ['Quaternary Deposits']
category = None
category2feature = {'Area': []}

count = 0
for i in range(len(data_type)):
    if type(data_type[i]) != str:
        category2feature[categories[count]].append(IDs[i])
    else:
        count += 1
        try:
            category2feature[categories[count]].append(IDs[i])
        except:
            category2feature[categories[count]] = [IDs[i]]

# Associate each feature to its category (defined with an ID)
features2idxcategory = {}
for i, cat in enumerate(categories):
    for feature in category2feature[cat]:
        features2idxcategory[feature] = i
df_catchment_properties.columns = IDs

df_catchment_names = df_catchment_properties[list(IDs[:5]) + ['H_MIN', 'H_MAX', 'H_MEAN']]  # retrieve some specific columns, why? RM

gis_id = df_catchment_properties['GIS_ID']
df_catchment_properties.drop(columns=['GIS_ID'], inplace=True)
df_catchment_properties.index = rows_idx

# Split main catchment information and attributes
df_infos = df_catchment_properties.iloc[:, 1:4]
df_data = df_catchment_properties.iloc[:, 4:]
df_data = df_catchment_properties.drop(df_data.columns[[3, 4]], axis = 1)

# Generate a unique ID for each catchment based on data source and numeric ID
betreiber2acronym = {'BAFU':'BAFU-',
                     'AARGAU':'AG',
                     'SOLOTHURN':'SO-',
                     'BERN':'BE-',
                     'BASEL LANDSCHAFT': 'BL-',
                     'LUZERN':'LU-',
                     'ZUERICH':'ZH-'}

df_catchment_names = df_catchment_names.dropna(subset=['org_ID'])
df_catchment_names['catchment_name'] = (
    get_acronym(df_catchment_names['Betreiber'].astype(str), betreiber2acronym)
    + df_catchment_names["org_ID"]
)
df_catchment_names.index = range(len(df_catchment_names['catchment_name']))
df_catchment_names['catchment_name'] = df_catchment_names['catchment_name'].apply(lambda x: x.replace('_','-'))

In [4]:
# Keep main catchment information for catchment with predefined ID only
df_catchment_names = df_catchment_names[df_catchment_names['GIS_ID'].apply(lambda x : (x in ALL_GIS_IDs))]

# PET

In [5]:
geolocator = Nominatim(user_agent='myapplication')

# Retrieve latitude from river name (why?)
latitudes = list(map(lambda x: get_latitude(geolocator, x), df_catchment_names['Gewaesser']))

dic_latitudes = {}
for i in range(len(df_catchment_names['Gewaesser'])):
    dic_latitudes[df_catchment_names['GIS_ID'].iloc[i]] = latitudes[i]

# Save dictionnary relating catchment ID with river latitude
with open('../data/real_data/dic_latitudes.pkl', 'wb') as handle:
    pickle.dump(dic_latitudes, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [6]:
# Add latitude to catchment ID
df_catchment_names['latitude'] = [dic_latitudes[GISID] for GISID in df_catchment_names['GIS_ID']]
print('Number of catchments: ', df_catchment_names.shape[0])
print('Add Latitudes')
df_catchment_names = df_catchment_names.drop(df_catchment_names[df_catchment_names['latitude'] == np.nan].index)
print('Number of catchments: ', df_catchment_names.shape[0])

# Filter catchment with data quality issues
locations_pb = df_info.dropna(subset=['INFO'])['GIS_ID'].to_numpy()  # select stations with a non NaN value in 'INFO', meaning there might be a quality problem
print('Remove catchments with data issues')
df_catchment_names = df_catchment_names[df_catchment_names['GIS_ID'].apply(lambda x: not(x in locations_pb))]
print('Number of catchments: ', df_catchment_names.shape[0])

Number of catchments:  1
Add Latitudes
Number of catchments:  1
Remove catchments with data issues
Number of catchments:  1


# Processing all stations

In [7]:
# Prepare a data file for each station/catchment of interest with all timeseries necessary to run GAMCR
for index, row in tqdm(df_catchment_names.iterrows()):
    GISID = row['GIS_ID']

    # Discharge data
    discharge = process_hydro(GISID, path_Q, df_catchment_names, path_daily)

    # Precipitation data
    precip = process_precip(GISID, path_J, path_daily)

    # Merge discharge and precipitation
    data = merging_dfs(precip, discharge)

    # Temperature
    Tmax = pd.read_csv(os.path.join(path_daily, 'Tmax', 'TmaxD_GIS_ID-{0}.csv').format(GISID), header=0,  engine='python')[['mean','datetime']]
    Tmax['datetime'] = Tmax['datetime'].apply(lambda x: datetime.strptime(str(x), '%Y-%m-%d'))
    Tmax = Tmax.rename(columns={'mean': 'tmax'})
    Tmax.sort_values(by=['datetime'])

    Tmin = pd.read_csv(os.path.join(path_daily, 'Tmin', 'TminD_GIS_ID-{0}.csv').format(GISID), header=0,  engine='python')[['mean','datetime']]
    Tmin['datetime'] = Tmin['datetime'].apply(lambda x: datetime.strptime(str(x), '%Y-%m-%d'))
    Tmin.sort_values(by=['datetime'])
    Tmin = Tmin.rename(columns={'mean': 'tmin'})

    Tabs = pd.read_csv(os.path.join(path_daily, 'Tabs', 'TabsD_GIS_ID-{0}.csv').format(GISID), header=0,  engine='python')[['mean','datetime']]
    Tabs['datetime'] = Tabs['datetime'].apply(lambda x: datetime.strptime(str(x), '%Y-%m-%d'))
    Tabs.sort_values(by=['datetime'])
    Tabs = Tabs.rename(columns={'mean': 'tabs'})

    data_daily = Tmin
    data_daily = pd.merge(data_daily, Tmax, on="datetime", how="inner")
    data_daily = pd.merge(data_daily, Tabs, on="datetime", how="inner")

    # PET
    lst = np.arange(0,len(data_daily),1)
    PET = np.array(list(map(lambda t: get_PET_hargreaves(data_daily.iloc[t]['tmin'], data_daily.iloc[t]['tabs'], data_daily.iloc[t]['tmax'], data_daily['datetime'][t], dic_latitudes[GISID]), lst) ))
    data_daily['pet'] = PET
    data_daily.set_index('datetime', inplace=True)
    data_hourly = data_daily.resample('h').ffill()
    df_treated_data = pd.merge(data, data_hourly, on="datetime", how="inner")

    # Export processed data
    path_data_export = os.path.join('..', 'data', 'real_data', 'GISID2hourly_data_withPET', f'{GISID}.csv')
    df_treated_data.to_csv(path_data_export, index=False, header=True) 

0it [00:00, ?it/s]

1it [00:05,  5.33s/it]
